In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import norm
import matplotlib as matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
## Sætter grænseværdien for animationsstørrelsen op##
matplotlib.rcParams['animation.embed_limit'] = 2**128

## Fjederpendul

Pendulets placering er fuldstændig defineret ved det polære koordinatsæt: $r,\phi$. Når fjederen har hvilelængden $L_0$, fåes da:

\begin{align}
\vec{r}(r,\phi)&=rsin(\phi)\hat{x}-rcos(\phi)\hat{y}\\
v^2&=\dot{r}^2+r^2\dot{\phi}^2\\
\end{align}
Med energiforholdende:
\begin{align}
T(\dot{r},\dot{\phi})&=\frac{m}{2}\big(\dot{r}^2+r^2\dot{\phi}^2\big)\\
U(r,\phi)&=\frac{k}{2}(r-L_0)^2+mgr(1-cos(\phi))
\end{align}
og heraf bestemmes Lagrangefunktionen som:
\begin{equation}
\mathcal{L}(r,\phi,\dot{r},\dot{\phi})=\dot{\phi})=\frac{m}{2}\big(\dot{r}^2+r^2\dot{\phi}^2\big)-\frac{k}{2}(r-L_0)^2-mgr(1-cos(\phi))
\end{equation}
hvoraf vi bestemmer de 2 bevægelsesligninger til:
\begin{align}
\ddot{r}&=r\dot{\phi}^2-\frac{k}{m}(r-L_0)+gcos(\phi)\\
\ddot{\phi}&=-\frac{g}{r}sin(\phi)-\frac{2}{r}\dot{r}\dot{\phi}
\end{align}

In [ ]:
## Konstanternes værdi ##
l0, k, m, g = 1, 40, 1, 9.82

##startbetingelser
r_0 = 1.3
r_prik_0 = 0
phi_0 = np.pi/2
phi_prik_0 = 0

yinit = [r_0,r_prik_0,phi_0,phi_prik_0]

tinit = 0
tfinal = 10
trange = [tinit,tfinal]
ts = np.linspace(tinit, tfinal, 1000)

def dydt(t,y):
    r = y[0]
    r_prik = y[1]
    phi = y[2]
    phi_prik = y[3]
    
    d_r_dt = r_prik
    d_phi_dt = phi_prik
    
    d_r_prik_dt = r*phi_prik**2-(k/m)*(r-l0)+g*np.cos(phi)
    d_phi_prik_dt = -(g/m)*np.sin(phi)-(2/r)*r_prik*phi_prik
    
    return [d_r_dt,d_r_prik_dt,d_phi_dt,d_phi_prik_dt]

mysol = solve_ivp(dydt, trange, yinit, t_eval = ts,rtol=3e-14)

ts = mysol.t
r, phi = mysol.y[0], mysol.y[2]
r_prik, phi_prik = mysol.y[1], mysol.y[3]

In [ ]:
plt.rc('font', size=16)
fig,ax = plt.subplots(2,2,figsize=(20,10))
ax[1][0].plot(r,r_prik,linestyle='--')
ax[1][1].plot(phi,phi_prik,linestyle='--')

ax[0][0].plot(ts,r)
ax[0][1].plot(ts,phi)

ax[1][0].grid()
ax[1][1].grid()
ax[0][0].grid()
ax[0][1].grid()

ax[1][0].set_xlabel(r'$r$')
ax[1][0].set_ylabel(r'$\frac{dr}{dt}=\dot{r}$')
ax[1][1].set_xlabel(r'$\phi$')
ax[1][1].set_ylabel(r'$\frac{d\phi}{dt}=\dot{\phi}$')


ax[0][0].set_xlabel('$t$')
ax[0][0].set_ylabel(r'$r$')
ax[0][1].set_xlabel('$t$')
ax[0][1].set_ylabel(r'$\phi$')


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

### Mass ###
mass_dot, = ax.plot([],[],'o',ms=8,color='green')

### Spring ###
spring, = ax.plot([],[],color='grey')     

### tracer ###
tracer, = ax.plot([],[],linewidth = 1, color='blue')

def angle_to_cartesian(dist,angle):
    x_vals = dist*np.sin(angle)
    y_vals = -dist*np.cos(angle)
    return x_vals, y_vals

def draw_spring(anchor_x0,anchor_y0,length,angle,nr_turns,spring_width,buffer_length):
    D, R, N, L = buffer_length, spring_width, nr_turns, length
    x1, y1 = anchor_x0, anchor_y0
    spring_length = L-2*D

## tegner fjeder i vandret mod +x ##
    xs = [x1,x1+D]
    ys = [y1,y1]
    
    k = 1
    for i in range(N-2):
        k*=-1
        if k<0:
            ys.append(y1+R)
        if k>0:
            ys.append(y1-R)
            
        x_temp = xs[i+1]+spring_length/N
        xs.append(x_temp)
        
    for j in range(2):
        xs.append(x1+spring_length+D*(j+1))
        ys.append(y1)
## roterer fjeder m. angle mod urets retning (0 grader = lodret) ##        
    c1 = -np.pi/2
    Rotation_matrix = np.array([[np.cos(c1+angle),-np.sin(c1+angle)],[np.sin(c1+angle),np.cos(c1+angle)]])
    final_xs, final_ys = [], []    
    for k in range(len(xs)):
        temp_vec = np.array([[xs[k]],[ys[k]]])
        final_vec = Rotation_matrix @ temp_vec
        final_xs.append(final_vec[0][0]), final_ys.append(final_vec[1][0])
        
    return final_xs, final_ys

def update(i):
    
    mass_x, mass_y = angle_to_cartesian(r[i],phi[i])
    mass_dot.set_data([mass_x],[mass_y])
    
    spring_x, spring_y = draw_spring(0,0,r[i],phi[i],20,0.125,0.3)
    spring.set_data([spring_x],[spring_y])
    
    tracer_x, tracer_y = angle_to_cartesian(r,phi)
    tracer.set_data(tracer_x[0:i+1],tracer_y[0:i+1])
    
    return mass_dot, spring,

## plotting anchor ##
ax.plot([0],[0],'o',ms=5,color="black")

## setting axes ## 
ax.set_xlim(-3,3), ax.set_ylim(-3,3), ax.set_aspect("equal")

## animating ## 
anim = animation.FuncAnimation(fig,
                               update,
                               frames=len(ts),
                               interval=10,
                               blit=True,
                               repeat_delay=0)
anim